In [1]:
import theano 
from theano import tensor as T

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
E:\Anac\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


## 符号变量

In [2]:
x = T.scalar(name='input', dtype='float32')  # 定义输入x
w = T.scalar(name='weight', dtype='float32')  # 定义权重w
b = T.scalar(name='bias', dtype='float32')  # 定义偏移量b
z = w * x + b # 输出z

In [3]:
# 编译程序
net_input = theano.function(inputs=[w, x, b], outputs=z)

In [4]:
print('net_input: %.2f'% net_input(2.0, 3.0, 0.5))

net_input: 6.50


### 1、使用内置的变量类型创建

In [5]:
# name指变量名称，dtype指变量的数据类型
x = T.scalar(name='input', dtype='float32')
data = T.scalar(name='data', dtype='float64')

### 2、自定义变量类型

In [6]:
# 因为内置的变量类型只能处理4维及以下的变量，所以想要处理高维的数据时，可以使用自定义变量类型

In [7]:
mytype = T.TensorType('float64', broadcastable=(), name=None, sparse_grad=False)
# broadcastable是布尔型数组，元组的等于变量的维度，如果为True，表示变量在对应维度上的数据可以广播，否侧就不可以

### 3、将Python数据类型或者Numpy数据变量转化为theano共享变量

In [8]:
import numpy as np

In [9]:
data  = np.array([[1,2], [3,4]])
shared_data = theano.shared(data)
type(shared_data)

theano.tensor.sharedvar.TensorSharedVariable

## 符号计算图模型

In [10]:
x = T.dmatrix('x')
y = T.dmatrix('y')
z =x + y

## 函数

In [11]:
x, y = theano.tensor.fscalars('x', 'y')
z1 = x + y
z2 = x * y

In [12]:
f = theano.function([x, y], [z1, z2])
print(f(2, 3))

[array(5., dtype=float32), array(6., dtype=float32)]


In [13]:
# 求导数
x = theano.tensor.fscalar('x')
y = 1 / (1 + theano.tensor.exp(-x))
dx = theano.grad(y, x)  # 偏导数函数
f = theano.function([x], dx)
print(f(3))  # 计算当x=3的时候，函数y的偏导数

0.04517666


In [14]:
# 更新共享变量参数
w = theano.shared(1)
x = theano.tensor.iscalar('x')
f = theano.function([x], w, updates=[[w, w+x]])
print(f(3))
print(w.get_value())

1
4


In [15]:
import numpy  as np
import theano  
import theano.tensor as T  
rng = np.random  

# 我们为了测试，自己生成10个样本，每个样本是3维的向量，然后用于训练 
N = 10     
feats = 3  
D = (rng.randn(N, feats).astype(np.float32), rng.randint(size=N, low=0, high=2).astype(np.float32))    

# 声明自变量x、以及每个样本对应的标签y(训练标签)  
x = T.matrix("x")  
y = T.vector("y")  

#随机初始化参数w、b=0，为共享变量  
w = theano.shared(rng.randn(feats), name="w")  
b = theano.shared(0., name="b")  

#构造代价函数
p_1 = 1 / (1 + T.exp(-T.dot(x, w) - b))   # s激活函数  
xent = -y * T.log(p_1) - (1-y) * T.log(1-p_1) # 交叉商代价函数
cost = xent.mean() + 0.01 * (w ** 2).sum()# 代价函数的平均值+L2正则项以防过拟合，其中权重衰减系数为0.01  
gw, gb = T.grad(cost, [w, b])             #对总代价函数求参数的偏导数  

prediction = p_1 > 0.5                    # 大于0.5预测值为1，否则为0.

train = theano.function(inputs=[x,y],outputs=[prediction, xent],updates=((w, w - 0.1 * gw), (b, b - 0.1 * gb)))#训练所需函数  
predict = theano.function(inputs=[x], outputs=prediction)#测试阶段函数  

#训练  
training_steps = 1000  
for i in range(training_steps):  
    pred, err = train(D[0], D[1])  
    print (err.mean())#查看代价函数下降变化过程  

0.9057049562046438
0.8854834960947194
0.8660582676562776
0.8474142734486476
0.8295349524531594
0.8124022821703345
0.7959969007843233
0.7802982468019452
0.765284713020927
0.7509338112705607
0.7372223441183882
0.7241265796573304
0.7116224255755463
0.6996855989517831
0.6882917885883176
0.6774168071615009
0.6670367310025563
0.6571280258844299
0.6476676577521209
0.6386331878665594
0.6300028523140406
0.6217556262492966
0.6138712735817597
0.6063303830785517
0.5991143920462372
0.5922055988721426
0.5855871657633298
0.5792431130268786
0.5731583061991343
0.5673184372640242
0.5617100011106012
0.5563202682757259
0.5511372549060536
0.5461496907597434
0.5413469859567817
0.5367191970806353
0.5322569931351999
0.5279516217710187
0.5237948761141429
0.5197790624599118
0.5158969690321508
0.5121418359552774
0.5085073265419611
0.5049874999615038
0.5015767853232372
0.4982699571841544
0.49506211246996185
0.49194864878304206
0.4889252440587821
0.4859878375228089
0.4831326118952961
0.4803559767842757
0.477654553

0.31987695605816924
0.3198425850144503
0.31980844263537006
0.3197745271724785
0.31974083689294847
0.3197073700794086
0.3196741250297805
0.31964110005711655
0.3196082934894395
0.31957570366958576
0.3195433289550488
0.3195111677178254
0.31947921834426385
0.3194474792349136
0.3194159488043774
0.31938462548116464
0.3193535077075467
0.31932259393941453
0.31929188264613717
0.3192613723104226
0.31923106142818053
0.31920094850838565
0.31917103207294406
0.3191413106565603
0.3191117828066068
0.31908244708299366
0.3190533020580414
0.3190243463163545
0.3189955784546966
0.31896699708186704
0.3189386008185794
0.31891038829734075
0.3188823581623331
0.3188545090692957
0.3188268396854087
0.31879934868917925
0.31877203477032723
0.31874489662967365
0.3187179329790301
0.3186911425410892
0.31866452404931656
0.3186380762478437
0.31861179789136307
0.3185856877450232
0.3185597445843256
0.3185339671950233
0.3185083543730198
0.3184829049242691
0.31845761766467834
0.3184324914200096
0.3184075250257845
0.31838271

0.31455939595436017
0.3145568017941366
0.314554219996685
0.31455165049512285
0.3145490932229711
0.3145465481141512
0.3145440151029825
0.31454149412417964
0.31453898511284994
0.31453648800449113
0.3145340027349884
0.31453152924061195
0.3145290674580147
0.31452661732422954
0.3145241787766672
0.3145217517531134
0.31451933619172656
0.3145169320310353
0.31451453920993633
0.3145121576676916
0.3145097873439264
0.31450742817862665
0.3145050801121366
0.31450274308515674
0.31450041703874126
0.31449810191429567
0.3144957976535748
0.31449350419868055
0.31449122149205927
0.3144889494764998
0.3144866880951313
0.3144844372914211
0.3144821970091719
0.31447996719252064
0.3144777477859354
0.3144755387342137
0.31447333998248056
0.31447115147618593
0.3144689731611027
0.31446680498332513
0.31446464688926584
0.31446249882565475
0.3144603607395363
0.31445823257826794
0.3144561142895176
0.31445400582126226
0.3144519071217856
0.314449818139676
0.31444773882382476
0.3144456691234242
0.3144436089879655
0.3144415

条件判断

In [16]:
from theano import tensor as T  
from theano.ifelse import ifelse  
import theano,time,numpy  

a,b=T.scalars('a','b')  
x,y=T.matrices('x','y')  
z_switch=T.switch(T.lt(a,b),T.mean(x),T.mean(y))#lt:a < b?  
z_lazy=ifelse(T.lt(a,b),T.mean(x),T.mean(y))  

#optimizer:optimizer的类型结构（可以简化计算，增加计算的稳定性）  
#linker:决定使用哪种方式进行编译(C/Python) 
f_switch = theano.function([a, b, x, y], z_switch,mode=theano.Mode(linker='vm'))  
f_lazyifelse = theano.function([a, b, x, y], z_lazy,mode=theano.Mode(linker='vm'))  

val1 = 0.  
val2 = 1.  
big_mat1 = numpy.ones((1000, 100))  
big_mat2 = numpy.ones((1000, 100))  

n_times = 10  

tic = time.clock()  
for i in range(n_times):  
    f_switch(val1, val2, big_mat1, big_mat2)  
print('time spent evaluating both values %f sec' % (time.clock() - tic))  

tic = time.clock()  
for i in range(n_times):  
    f_lazyifelse(val1, val2, big_mat1, big_mat2)  
print('time spent evaluating one value %f sec' % (time.clock() - tic))

time spent evaluating both values 0.008168 sec
time spent evaluating one value 0.006623 sec


E:\Anac\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
E:\Anac\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
E:\Anac\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
E:\Anac\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


循环语句

In [17]:
import theano
import theano.tensor as T
import numpy as np

# 定义单步的函数,实现a*x^n
# 输入参数的顺序要与下面scan的输入参数对应
def one_step(coef, power, x):
    return coef * x ** power

coefs = T.ivector()  # 每步变化的值,系数组成的向量
powers = T.ivector() # 每步变化的值,指数组成的向量
x = T.iscalar()      # 每步不变的值,自变量

# seq,out_info,non_seq与one_step函数的参数顺序一一对应
# 返回的result是每一项的符号表达式组成的list
result, updates = theano.scan(fn = one_step,
                       sequences = [coefs, powers],
                       outputs_info = None,
                       non_sequences = x)

# 每一项的值与输入的函数关系
f_poly = theano.function([x, coefs, powers], result, allow_input_downcast=True)

coef_val = np.array([2,3,4,6,5])
power_val = np.array([0,1,2,3,4])
x_val = 10

print("多项式各项的值: ",f_poly(x_val, coef_val, power_val))
#scan返回的result是每一项的值，并没有求和，如果我们只想要多项式的值，可以把f_poly写成这样：
# 多项式每一项的和与输入的函数关系
f_poly = theano.function([x, coefs, powers], result.sum(), allow_input_downcast=True)

print("多项式和的值：",f_poly(x_val, coef_val, power_val))

多项式各项的值:  [    2    30   400  6000 50000]
多项式和的值： 56432


In [18]:
import theano
import theano.tensor as T
from theano import shared
import numpy as np

#定义一个共享变量，并初始化为0
state = shared(0)
inc = T.iscalar('inc')
accumulator = theano.function([inc], state, updates=[(state, state+inc)])
# 打印state的初始值
print(state.get_value())
accumulator(1) # 进行一次函数调用
# 函数返回后，state的值发生了变化
print(state.get_value()) 

0
1
